# Gremlin Workshop - Section 06: Profile and Query Explain
Troubleshooting query performance is something every graph database developer will need to do at some point.  Thankfully, both Gremlin and Neptune provide a couple of different mechanisms for seeing how a query is executed.  In the following section we will cover different methods for exposing query execution timing and performance.

Happy Traversing!

## Graph Model

As we have in previous notebooks, we've included here a copy of the graph data model for this dataset as reference to aid in writing your queries.

<img src="https://ee-assets-prod-us-east-1.s3.amazonaws.com/modules/f3f89ef4607743429fb01ae23d983197/v1/workshop/images/datamodel-updated.png"/>


### 06.01 - Using Gremlin Profile

As part of the Apache Tinkerpop framework, the Gremlin language has a native command called `profile()` which provides a high-level overview of how a given traversal is executed on the underlying supported database.  Within a `profile()` output, you can see timings for each step in a traversal along with the cardinality (or number of vertices/edges) touched as part of that step.  This helps us troubleshoot performance as the trick to any performant graph traversal is to "touch" as little of the graph as possible to return your desired result.

Let's try an example of this.  Say we wanted to find all genre's of film that Harrison Ford has played in.  A typical relational database developer may think of a query that looks like this (below).  Here, we start with the genre vertices and use a where clause to filter out only the genre vertices that have relationships across movies to Harrison Ford.  The following two code blocks show the output and the query profile:

In [ ]:
%%gremlin

g.V().hasLabel('Genre').where(in('genre').out('actor').has('name','Harrison Ford')).dedup()

In [ ]:
%%gremlin

g.V().hasLabel('Genre').where(in('genre').out('actor').has('name','Harrison Ford')).dedup().profile()

In the query above, we looked at all genre vertices, all movie vertices, and all actor vertices. It ran relatively quickly, but how can we narrow the focus of our query? What if we tried this from the other direction, starting with Harrison Ford's vertex and going the other way? Think of your queries/traversals as funnels. You want to start off with as few vertices/edges as possible (and cross as few as possible) to get to your result.  

Give this a try here:

In [ ]:
%%gremlin

g.V().hasLabel('Artist').has('name','Harrison Ford').in('actor').out('genre').dedup()

In [ ]:
%%gremlin

g.V().hasLabel('Artist').has('name','Harrison Ford').in('actor').out('genre').dedup().profile()

That was MUCH faster!  It is because we had to evaluate a lot less data.  We had one Artist vertex (Harrison Ford) and only the movies related to Harrison Ford.  Subsequently, only the genre vertices related to those movies.

### 06.02 - Using Gremlin Explain and Profile APIs

Profile is a great tool to use, but it doesn't necessarily tell the entire story of what Neptune is doing under-the-covers.  Inside of Neptune is a query parser and optimizer that we at AWS have custom built for both the Apache TinkerPop and RDF graph frameworks and their query languages (Gremlin and SPARQL respectively).  In order to expose a little more about what the optimizer is doing, we've created a REST API that you can use to evaluate your queries.  Below is an example of how to use the Gremlin Query Explainer in Neptune.

Additional Documentation:
- Using the Gremlin Explain API in Neptune - https://docs.aws.amazon.com/neptune/latest/userguide/gremlin-explain-api.html
- Profile API - https://docs.aws.amazon.com/neptune/latest/userguide/gremlin-profile-api.html

Here we can pass our original queries to the /explain API.  You'll notice that results come back immediately.  With this, /explain is returning an execution plan but it __IS NOT__ executing the query.

In [ ]:
%%gremlin explain

g.V().hasLabel('Genre').where(in('genre').out('actor').has('name','Harrison Ford')).dedup()

In [ ]:
%%gremlin explain

g.V().hasLabel('Artist').has('name','Harrison Ford').in('actor').out('genre').dedup()

You'll notice in the two outputs above, that the Optimized Traversals are different.  This is why we see drastically different performance between the two traversals.  We can take a deeper look at this using the Neptune Profile API and actually running out queries.  The profile API will return a total number of objects "touched" in order to complete our traversal and return results.

Below we have the API commands to run /profile against both the poorly performing traversal and the more optimal traversal.  Run these now:

In [ ]:
%%gremlin profile

g.V().hasLabel('Genre').where(in('genre').out('actor').has('name','Harrison Ford')).dedup()

In [ ]:
%%gremlin profile

g.V().hasLabel('Artist').has('name','Harrison Ford').in('actor').out('genre').dedup()

At the bottom of the /profile output, you'll see a section called "Index Operations".  Here you see the counts on number of objects (vertices and edges) that were traversed to complete the query.  You'll notice that the number traversed in the first traversal is many orders of magnitude larger than the second query.  This is something to keep in mind as you write performant graph traversals.

### Review

In the preceding noteboook we covered how to use both the native Gremlin profile() functionality and Neptune's profile and explain APIs to look at the performance of queries.  These can be used during the process to optimize queries for better performance.

### End of Section 06 - Move onto the next notebook.